In [41]:
%%writefile bizcardx.py
import pandas as pd
import streamlit as st
import easyocr
import sqlite3
from sqlite3 import Error
from streamlit_option_menu import option_menu
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt
import re

# SETTING PAGE CONFIGURATIONS
ii="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFwAAABcCAMAAADUMSJqAAAA21BMVEXw8O8hlvPi6gBFWmTqAABHie0A4NMAkfP18++rzfFGWF87bpkjlO3h6PD///sAjvPx9vXx8fVAhu04guy2zPX1/Pvn7U3tTU3wdHTwgIBnrvXE2/iNse709tXx9Kj0qKjj5ORrnO4sR1QAh/I+nvPK3PBWp/L19t7l9vT88/OI7OQdYZP46un1y8z2t7f3wsKy8ezV9fLuV1fsMjL23NvsISCd7udj597tQUHvZmZI5dryjIzs8X3y9MHw87D29unw85zl7C/znJx8uPWevPN/qfHt8YtalO7q8G+h1Iu8AAADKklEQVRoge3Y/1uaQBwHcGSp1xYKIrZlpjZRsdxwOV1kiput/v+/aMBxxJfPcXestufZeD/9UuHL475wH06SypT5L6OmwvMZhLhk43r+qR/PZ8Tkram9sheqxbjMmH85SudmybLtWpDZNI9Xr08ztJevRj4+rZGs6Lq6hGgvy9yOsVYRXlvTdPWaYh/N8/FZja0bNzScMVKLGF6zwYvVPs3uM/rcWsd18C7VqOHfTuO5nTNsf7o4Tv6gRqM5T64hg2MVWRZSybA6AB71Sv7UyPkCMq5TAL/D9i2zE2gh032R/ZfxHuMfMK75EcNVXlybbDpPPTGdF9fuZT9bIZ0XRzLO7hXwsOGCTSdrCZhtv49LqkNpeLJbdiE+EbE93Z6twGd6ckDdwN4ITkZ/qYJ/Tk3F7V6WL0RtZHHh3iJC3g9XnpttOzUHfOSmcG8/7x5zxSR8uCNBegpHZqXOmYqJdfJsYUxF365XuFPHuk1fROSpeIfxMb9dqYyZ+Dx8nAe/KQIN95quMLpFMr7H9kulKYI3FcaAevqy31+Gc2Vw9U4gV4PgQ5a9dtYLauUS3dHg8q1ALgfhxyyJVS9i/I1AIpwvJf438EKzhS9K80QgeBEJ4OIrtMRfAq+Pwpy8An7WIBm9PD5SSF6h5ZWPJPkbU7E+H5/FQ+0bLvy5wAnw+o9GMrRxjXD62whS2m0FxVt+0k3kmNVy7d7dSiCP2rqXNorh6bqIgWudoDIGdGTqVS86vq5IaaH9DErjDoQfMH4Iml6kKEJ7alGPzls+3jrHF4qUc01czpEXqR4LL1KI8uP8JXSXTGAB3OfFin8hXDQlXuIvgjf+RMsPxfE9HT9g/KEwnnPU4e0V1UBXCurh4xw+XlIwXq2a1JODJJYMckMb2iwk9bGFcf1xGM+DCeqa20kk7HBZdiGc9IvXM8mQfTVpd2RK4BttkKano2dfuLULmk05AUJKFdb1bMdoTxT7iV64wDqEbyg2/ZQZKUM9y7eqjWy39EAbHMwoDXPY0lN5hGa+5mbk/WbHOHFDSDPbiXQ1eCpOthfx3PcQz2Ee3xrKrCIOuUyZfzG/AHRNgdlz5qz0AAAAAElFTkSuQmCC"

st.set_page_config(
    page_title=" Extracting Business Card Data with OCR ",
    page_icon=ii,
    layout="wide",
    initial_sidebar_state="expanded")

# Add custom CSS to adjust the layout
st.markdown(
    """
    <style>
    .stApp {
        display: flex;
        flex-direction: column;
        align-items: flex-start;
    }
    .stApp > div:first-child {
        order: -1;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Create a container for the icon and title
icon_url = "https://cdn-icons-png.freepik.com/256/1278/1278649.png?ga=GA1.1.754611615.1706614838"
title = "Extracting Business Card Data with OCR"
st.markdown(
    f'<div><img src="{icon_url}" alt="Icon" style="height: 50px; margin-right: 10px;">{title}</div>',
    unsafe_allow_html=True
)

#Title

st.title("Extracting Business Card Data with OCR by :green[vk] ")

# SETTING-UP BACKGROUND IMAGE
def setting_bg():
    st.markdown(f"""
    <style>
        .stApp {{
            background: linear-gradient(to right, #ffffff, #e0e0e0);
            background-size: cover;
            transition: background 0.5s ease;
        }}
        h1, h2, h3, h4, h5, h6 {{
            color: #333333; /* Dark gray text color */
            font-family: 'Roboto', sans-serif;
        }}
        .stButton>button {{
            color: #333333; /* Dark gray text color */
            background-color: #ffffff; /* White button background */
            transition: all 0.3s ease-in-out;
        }}
        .stButton>button:hover {{
            color: #ffffff; /* White text color on hover */
            background-color: #666666; /* Dark gray background on hover */
        }}
        .stTextInput>div>div>input {{
            color: #333333; /* Dark gray text color */
            background-color: #ffffff; /* White input background */
        }}
    </style>
    """, unsafe_allow_html=True)
setting_bg()


SELECT = option_menu(
    menu_title=None,
    options=["Home", "Upload and Extract and Store", "Modify"],
    icons=["bar-chart", "toggles", "at"],
    default_index=2,
    orientation="horizontal",
    styles={
        "container": {"padding": "0!important", "background-color": "white", "size": "cover"},
        "icon": {"color": "black", "font-size": "20px"},
        "nav-link": {"font-size": "20px", "text-align": "center", "margin": "-2px", "--hover-color": "#6F36AD"},
        "nav-link-selected": {"background-color": "#6F36AD"}
    }

)


# INITIALIZING THE EasyOCR READER
reader = easyocr.Reader(['en'])

# CONNECTING WITH SQLite DATABASE
conn = None
try:
    conn = sqlite3.connect("business_cards.db")
    cur = conn.cursor()
except Error as e:
    st.error(f"Error connecting to SQLite database: {e}")

# TABLE CREATION
cur.execute('''CREATE TABLE IF NOT EXISTS Business_Cards
                   (id INTEGER PRIMARY KEY AUTOINCREMENT,
                    company_name VARCHAR(50),
                    card_holder VARCHAR(50),
                    designation VARCHAR(50),
                    mobile_number VARCHAR(50),
                    email TEXT,
                    website TEXT,
                    area TEXT,
                    city VARCHAR(50),
                    state VARCHAR(50),
                    pin_code VARCHAR(10),
                    image BLOB
                    )''')

# HOME MENU
if SELECT == "Home":

    st.markdown(":black_square_button: **Project Title** : BizCardX: Extracting Business Card Data with OCR")

    technologies = "streamlit GUI, SQLite, Data Extraction ,EasyOCR"
    st.markdown(f":black_square_button: **Technologies** : {technologies}")

    overview = "Streamlit application that allows users to upload an image of a business card and extract relevant information from it using easyOCR."
    st.markdown(f":black_square_button: **Overview** : {overview}")
    col1,col2,col3=st.columns(3)
    with col1:
      icon_url = "https://miro.medium.com/v2/resize:fit:602/1*-JvkJ2KeG61PTrAqjpsFLw.png"
      st.image(icon_url,use_column_width=True)
    with col2:
      name = "**VINOTH KUMAR**"
      d_link = "https://play.google.com/store/apps/details?id=com.phonepe.app&hl=en_IN&gl=US"
      mail = f'Mail: **vinoharish8799@gmail.com**'
      social_media = {"GITHUB": "https://github.com/vinothkumarpy",
                      'LINKED_IN': 'https://www.linkedin.com/in/vinoth-kumar-s-370724281/'}
      st.markdown(f"# {name}")
      st.subheader("An Aspiring Data_Analyst")
      st.markdown(mail)
      st.markdown("## Social Media")
      for platform, link in social_media.items():
        st.markdown(f"[{platform}]({link})")
    with col3:
      image_url = 'https://media.licdn.com/dms/image/D5603AQEdUYVxth10vA/profile-displayphoto-shrink_200_200/0/1696432308722?e=1712188800&v=beta&t=XVFpGzex8677InWdrqvpg9Oetr3lgD7d9te4yQ9VFKU'
      st.image(image_url, width=400)
# Initialize df as None
df = None

# UPLOAD AND EXTRACT MENU
if SELECT == "Upload and Extract and Store":
    tab1, tab2, tab3 = st.tabs(["UPLOAD","EXTRACT","STORE"])
    with tab1:
        st.markdown("### Upload a Business Card")
        uploaded_card = st.file_uploader("Upload here", label_visibility="collapsed", type=["png", "jpeg", "jpg"])

        if uploaded_card is not None:
            # Save the uploaded file to a temporary directory
            temp_dir = os.path.join("uploaded_cards")
            os.makedirs(temp_dir, exist_ok=True)
            temp_file_path = os.path.join(temp_dir, "temp_card.jpg")
            with open(temp_file_path, "wb") as f:
                f.write(uploaded_card.getbuffer())

            # Display the uploaded card
            st.markdown("### You have uploaded the card")
            st.image(uploaded_card)

    with tab2:
        if uploaded_card is not None:
            if hasattr(uploaded_card, 'name') and uploaded_card.name is not None:
                with open(os.path.join("uploaded_cards", uploaded_card.name), "wb") as f:
                    f.write(uploaded_card.getbuffer())
                # Rest of the code that depends on uploaded_card
                image = cv2.imread(temp_file_path)
                res = reader.readtext(temp_file_path)
                st.markdown("### Image Processed and Data Extracted")

                def image_preview(image, res):
                    for (bbox, text, prob) in res:
                       # unpack the bounding box
                        (tl, tr, br, bl) = bbox
                        tl = (int(tl[0]), int(tl[1]))
                        tr = (int(tr[0]), int(tr[1]))
                        br = (int(br[0]), int(br[1]))
                        bl = (int(bl[0]), int(bl[1]))
                        cv2.rectangle(image, tl, br, (0, 255, 0), 2)
                        cv2.putText(image, text, (tl[0], tl[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    plt.rcParams['figure.figsize'] = (15, 15)
                    plt.axis('off')
                    plt.imshow(image)

                # DISPLAYING THE CARD WITH HIGHLIGHTS
                with st.spinner("Please wait, processing image..."):
                        st.set_option('deprecation.showPyplotGlobalUse', False)
                        saved_img = os.path.join(os.getcwd(), "uploaded_cards", uploaded_card.name)
                        image = cv2.imread(saved_img)
                        res = reader.readtext(saved_img)
                        st.pyplot(image_preview(image, res))

               # easyOCR
                saved_img = os.path.join(os.getcwd(), "uploaded_cards", uploaded_card.name)
                result = reader.readtext(saved_img, detail=0, paragraph=False)

                # CONVERTING IMAGE TO BINARY TO UPLOAD TO SQL DATABASE
                def img_to_binary(file):
                    # Convert image data to binary format
                    with open(file, 'rb') as file:
                        binaryData = file.read()
                    return binaryData

                data = {"company_name": [],
                        "card_holder": [],
                        "designation": [],
                        "mobile_number": [],
                        "email": [],
                        "website": [],
                        "area": [],
                        "city": [],
                        "state": [],
                        "pin_code": [],
                        "image": img_to_binary(saved_img)
                        }

                def get_data(res):
                     for ind, i in enumerate(res):
                        # To get WEBSITE_URL
                        if "www " in i.lower() or "www." in i.lower():
                               data["website"].append(i)
                        elif "WWW" in i:
                               data["website"] = res[4] + "." + res[5]
                        # To get EMAIL ID
                        elif "@" in i:
                               data["email"].append(i)
                        # To get MOBILE NUMBER
                        elif "-" in i:
                             data["mobile_number"].append(i)
                        if len(data["mobile_number"]) == 2:
                             data["mobile_number"] = " & ".join(data["mobile_number"])
                        # To get COMPANY NAME
                        elif ind == len(res) - 1:
                            data["company_name"].append(i)
                        # To get CARD HOLDER NAME
                        elif ind == 0:
                            data["card_holder"].append(i)
                        # To get DESIGNATION
                        elif ind == 1:
                            data["designation"].append(i)
                        # To get AREA
                        if re.findall('^[0-9].+, [a-zA-Z]+', i):
                            data["area"].append(i.split(',')[0])
                        elif re.findall('[0-9] [a-zA-Z]+', i):
                            data["area"].append(i)
                        # To get CITY NAME
                        match1 = re.findall('.+St , ([a-zA-Z]+).+', i)
                        match2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
                        match3 = re.findall('^[E].*', i)
                        if match1:
                            data["city"].append(match1[0])
                        elif match2:
                            data["city"].append(match2[0])
                        elif match3:
                            data["city"].append(match3[0])
                        # To get STATE
                        state_match = re.findall('[a-zA-Z]{9} +[0-9]', i)
                        if state_match:
                            data["state"].append(i[:9])
                        elif re.findall('^[0-9].+, ([a-zA-Z]+);', i):
                            data["state"].append(i.split()[-1])
                        if len(data["state"]) == 2:
                            data["state"].pop(0)
                        # To get PINCODE
                        if len(i) >= 6 and i.isdigit():
                            data["pin_code"].append(i)
                        elif re.findall('[a-zA-Z]{9} +[0-9]', i):
                            data["pin_code"].append(i[10:])
                get_data(result)

               # FUNCTION TO CREATE DATAFRAME
                def create_df(data):
                    df = pd.DataFrame(data)
                    return df
                df = create_df(data)
                st.success("### Data Extracted!")
                st.write(df)
                pass

# STORING data in SQLite
    with tab3:
        if df is not None and not df.empty:
            try:
                for i, row in df.iterrows():
                    sql = """INSERT INTO Business_Cards(company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image)
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
                    cur.execute(sql, tuple(row))
                    conn.commit()
                st.success("Data uploaded to the database successfully!")
            except Error as e:
                st.error(f"Failed to store data in the database: {e}")
        else:
            st.warning("No data to store. Please upload and extract business card data first.")
            pass

# MODIFY MENU
if SELECT == "Modify":
    col1, col2, col3 = st.columns([3, 3, 2])
    col2.markdown("## Alteration and Deletion of Datas")
    column1, column2 = st.columns(2, gap="large")

    try:
        with column1:
            cur.execute("SELECT card_holder FROM Business_Cards")
            result = cur.fetchall()
            business_cards = {row[0]: row[0] for row in result}

            selected_card = st.selectbox("Select a card holder name to update", list(business_cards.keys()))
            st.markdown("#### Modify the Datas")

            cur.execute("SELECT company_name, card_holder, designation, mobile_number, email, website, area, city, state, pin_code FROM Business_Cards WHERE card_holder=?", (selected_card,))
            result = cur.fetchone()

            # DISPLAYING ALL THE INFORMATION
            company_name = st.text_input("Company Name", result[0])
            card_holder = st.text_input("Card Holder", result[1])
            designation = st.text_input("Designation", result[2])
            mobile_number = st.text_input("Mobile Number", result[3])
            email = st.text_input("Email", result[4])
            website = st.text_input("Website", result[5])
            area = st.text_input("Area", result[6])
            city = st.text_input("City", result[7])
            state = st.text_input("State", result[8])
            pin_code = st.text_input("Pin Code", result[9])

            if st.button("Commit changes to DB"):
                # Update the information for the selected business card in the database
                cur.execute("""
                    UPDATE Business_Cards
                    SET company_name=?, card_holder=?, designation=?, mobile_number=?, email=?, website=?, area=?, city=?, state=?, pin_code=?
                    WHERE card_holder=?
                """, (company_name, card_holder, designation, mobile_number, email, website, area, city, state, pin_code, selected_card))
                conn.commit()
                st.success("New Information updated in the database successfully.")

        with column2:
            cur.execute("SELECT card_holder FROM Business_Cards")
            result = cur.fetchall()
            business_cards = {row[0]: row[0] for row in result}

            selected_card = st.selectbox("Select a card holder name to delete", list(business_cards.keys()))
            st.write(f"### You have selected :green[**{selected_card}'s**] card to delete")
            st.write("#### Are You Sure")

            if st.button("Yes,sure"):
                cur.execute(f"DELETE FROM Business_Cards WHERE card_holder=?", (selected_card,))
                conn.commit()
                st.success("Business card infos deleted from the database.")
    except Error as e:
        st.warning(f"Error processing database: {e}")

    if st.button("View updated data"):
        cur.execute("SELECT company_name, card_holder, designation, mobile_number, email, website, area, city, state, pin_code FROM Business_Cards")
        updated_data = pd.DataFrame(cur.fetchall(), columns=["Company Name", "Card Holder", "Designation", "Mobile Number", "Email", "Website", "Area", "City", "State", "Pin Code"])
        st.write(updated_data)

# Close the database connection
if conn is not None:
    conn.close()

Overwriting bizcardx.py


In [ ]:
!streamlit run /content/bizcardx.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.237.18.84
npx: installed 22 in 3.059s
your url is: https://young-glasses-wash.loca.lt
